<a href="https://colab.research.google.com/github/dobrevajovana/QWEN-VL-video-QA/blob/main/Qwen2_VL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color = blue>QWEN for detectine marinetime objects⛵</font>

For this example we are using the free source dataset [MODD](https://www.vicos.si/resources/modd/) (Maritime Object Detection Dataset)
This dataset contains annotated images and videos of maritime objects, such as ships and buoys, to facilitate object detection and classification tasks.

# 1. Installing Qwen2-VL

In [1]:
!nvidia-smi

Sun Oct 20 13:13:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate

  Cloning https://github.com/huggingface/transformers (to revision 21fac7abba2a37fae86106f87fcf9974fd1e3830) to /tmp/pip-req-build-3fjjsnpw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3fjjsnpw
  Running command git rev-parse -q --verify 'sha^21fac7abba2a37fae86106f87fcf9974fd1e3830'
  Running command git fetch -q https://github.com/huggingface/transformers 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Running command git checkout -q 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Resolved https://github.com/huggingface/transformers to commit 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9677030 sha256=279dd6fb22622b8a9ff83f0e0be33e1483da70a861b7ca50e63ad7ed2c971a3e
  Stored in directory: /root/.cache/pip/

Utility library for a more smooth work

In [3]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 17.4 MB/s eta 0:00:00


# 2. Video set up

Qwen2-VL can be used for Q&A, image understanding, video understanding and more. Refer to the official [Github repository](https://github.com/QwenLM/Qwen2-VL) for more details.

Here we'll focus specifically on video.

We'll work with this video and frames, find it on:
- https://drive.google.com/drive/folders/1gmuEUdroFIcHAWnL5dJSLsrl1KNE__qM?usp=sharing

# 3. Qwen2-VL for Video Understanding

In [4]:
import torchvision
import transformers
import torch

print(transformers.__version__)

4.45.0.dev0


In [5]:
!pip uninstall transformers -q
!pip install git+https://github.com/huggingface/transformers -q

Proceed (Y/n)? y
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.0 MB/s eta 0:00:00


##🛑 Restart Runtime

In [7]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)

"""
# flash_attention_2 is suggested for video scenarios
model = Qwen2VLForConditionalGeneration.from_pretrained(
     "Qwen/Qwen2-VL-2B-Instruct",
     torch_dtype=torch.bfloat16,
     attn_implementation="flash_attention_2",
     device_map="auto",
)"""

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

We create a function to retrieve a portion of the frames in the video

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from math import ceil

def get_frame_list(output_path, fraction=0.0125):
    # Get all jpg files in the directory
    all_frames = sorted([f for f in os.listdir(output_path) if f.endswith('.jpg')])

    total_frames = len(all_frames)
    frames_to_keep = ceil(total_frames * fraction)

    # Calculate the step size to evenly distribute the selected frames
    step = total_frames // frames_to_keep

    # Select the frames
    selected_frames = all_frames[::step][:frames_to_keep]

    # Create the full paths for the selected frames
    frame_paths = [f"file://{os.path.join(output_path, frame)}" for frame in selected_frames]

    return frame_paths

In [20]:
import markdown
import textwrap
def query_video(prompt, use_frames=True, frames_path="/content/drive/MyDrive/Phd research/SAILOR/07/images", video_path=None):
    if use_frames:
        # Get the frames
        selected_frames = get_frame_list(frames_path)

        # Create messages structure for frames
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "video",
                        "video": selected_frames,
                        "fps": 1.0,
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ]
    else:
        # Create messages structure for the entire video
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "video",
                        "video": f"file://{video_path}",
                        "max_pixels": 360 * 420,
                        "fps": 1.0,
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ]

    print(f"Using {'frames' if use_frames else 'entire video'} for inference.")

    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference
    with torch.no_grad():  # Use no_grad to save memory during inference
        generated_ids = model.generate(**inputs, max_new_tokens=128)

    # Trim the generated output to remove the input prompt
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    # Decode the generated text
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    print(textwrap.fill(output_text[0]))
    torch.cuda.empty_cache()

Let's do inference ⚒ 🙂

#3. Video understanding (using frames)

In [21]:
query_video("What are the objects that you see in surrounding",
            use_frames=True, video_path="/content/drive/MyDrive/Phd research/SAILOR/07/gtvideo.avi")

Using frames for inference.
The objects visible in the surrounding area include:  1. **Boat**: A
small motor boat is visible in the water, with two people on board. 2.
**Buoy**: There are two yellow buoys floating in the water. 3.
**Cityscape**: In the background, there is a cityscape with buildings
and possibly a mountain or hillside. 4. **Cloudy Sky**: The sky is
cloudy, indicating overcast weather.  These objects suggest a coastal
or lakeside setting with a boat and buoys, possibly near a city or
town.


In [22]:
query_video("What is the approximate distance to the landscape?",
            use_frames=True, video_path="/content/drive/MyDrive/Phd research/SAILOR/07/gtvideo.avi")

Using frames for inference.
The approximate distance to the landscape is about 100 meters.


In [23]:
query_video("Is there a risk of hitting the buoys?",
            use_frames=True, video_path="/content/drive/MyDrive/Phd research/SAILOR/07/gtvideo.avi")

Using frames for inference.
There is no existence of any risk of hitting the buoys in the given
information.
